In [ ]:
from sql import Sql
import pandas
import pyodbc

In [10]:
def get_db_con():
    # make a connection string
    return pyodbc.connect('Trusted_Connection=yes',
                             driver='{ODBC Driver 13 for SQL Server}',
                             server='localhost',
                             database='Weather')

In [11]:
# get a connection to the database
con = get_db_con()
cursor = con.cursor()

In [12]:
# get an instance of the SQL query generator
sql = Sql()

In [13]:
# load the weather data into pandas
df = pandas.read_json('vancouverweather.json')

In [46]:
# parse out the clouds data and format to remove the dictoinary in the clouds column
# so the resulting rows are each "percent", "LinuxTime"
df_clouds = df.loc[:, 'clouds':'dt']
df_clouds['clouds'] = [v['all'] for v in df_clouds['clouds']]

In [47]:
# set up columns for sql insert
columns = ['"percent"', 'LinuxTime']
query = sql.get_insert(table_name="weather.clouds", columns=columns)

In [48]:
# loop over our clouds data frame and prep for db insert
for index, row in df_clouds.iterrows():
    values = [int(row['clouds']), str(row['dt'])]
    cursor.execute(query, values)

In [49]:
# commit the clouds insert transactions
cursor.commit()

In [20]:
# next we'll parse out the main weather stats buy slicing the main df from dt to main
# and then dropping the dt_iso column that is in between
df_main = df.loc[:, 'dt':'main']
df_main = df_main.drop('dt_iso', 1)

In [30]:
# next we'll parse out each key value that is burried in the "main" column
df_main['temp'] = [r['temp'] for r in df_main['main']]
df_main['min_temp'] = [r['temp_min'] for r in df_main['main']]
df_main['max_temp'] = [r['temp_max'] for r in df_main['main']]
df_main['pressure'] = [r['pressure'] for r in df_main['main']]
df_main['humidity'] = [r['humidity'] for r in df_main['main']]

In [32]:
# now that we've parsed out the main info, we can ditch the column
df_main = df_main.drop('main', 1)

In [43]:
# set up the column names for sql server insert
columns = ['LinuxTime', 'temp', 'min_temp', 'max_temp', 'pressure', 'humidity']
query = sql.get_insert('weather.main', columns)

In [44]:
# loop through our main records to insert to db
for index, row in df_main.iterrows():
    values = [int(row['dt']), 
              float(row['temp']), 
              float(row['min_temp']), 
              float(row['max_temp']), 
              int(row['pressure']), 
              int(row['humidity'])]
    cursor.execute(query, values)

In [45]:
cursor.commit()

In [115]:
df_weather = df.loc[:, 'dt': 'weather']
df_weather = df_weather.drop(['dt_iso', 'main', 'rain', 'snow'], 1)
df_weather2 = pandas.DataFrame(columns=['dt', 'id', 'main', 'description', 'icon'], dtype=['int', 'int', 'string', 'string', 'string'])

TypeError: data type not understood

In [113]:
for index, row in df_weather.iterrows():
    for item in row['weather']:
        df_weather2 = df_weather2.append({'dt': row['dt'],
                                          'id': item['id'],
                                          'main': item['main'],
                                          'description': item['description'],
                                          'icon': item['icon']}, 
                                         ignore_index=True)
          
        # df_weather2['dt'] = row['dt']
        # df_weather2['id'] = item['id']
        # df_weather2['main'] = item['main']
        # df_weather2['description'] = item['description']
        # df_weather2['icon'] = item['icon']

KeyboardInterrupt: 

In [114]:
df_weather2

,dt,id,main,description,icon
0,1.349096e+09,NaN,NaN,NaN,NaN
1,1.349096e+09,NaN,NaN,NaN,NaN
2,1.349096e+09,501.0,Rain,moderate rain,10n
3,1.349096e+09,701.0,Mist,mist,50n
4,1.349186e+09,800.0,Clear,sky is clear,02d
5,1.349190e+09,800.0,Clear,sky is clear,02d
6,1.349194e+09,800.0,Clear,sky is clear,02d
7,1.349197e+09,800.0,Clear,sky is clear,02d
8,1.349201e+09,800.0,Clear,sky is clear,02d
9,1.349204e+09,800.0,Clear,sky is clear,02d
